# Explore whether the Zillow Data has the accurate substation attached

The substation attachment to the Zillow house data was completed before the  start of this project. This repository explores the accuracy of the pre-assingments. The pre-assignments are also in a csv as opposed to in the geo-spatial format. This repository also explores any potential differences between the csv data and the geo-spatial Zillow data. 

In [1]:
# load necessary libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point # to make the lat and long of the zillow data into a gpd object
from shapely.geometry import box  # To create polygon bounding box
import glob
import folium

In [20]:
# SDGE substation polygons
sdge_substations = gpd.read_file("../../../../capstone/electrigrid/data/utilities/sandiego_grid/substation_sdge.geojson")

In [21]:
sdge_substations.head()

,OBJECTID,NAME,FACILITYID,SUBSTATIONTYPE,DISTRICT,IMAP_VOLTAGE,IMAP_LOAD_PROFILE,EXIST_GEN,QUE_GEN,TOT_GEN,PROJ_LOAD,PENETRATION,geometry
0,24321,CANNON,CAN,138/12 kV,NC,12kV,(576 Data Points),44.73,3.79,48.53,47.12,37.0,"POLYGON ((-117.35695 33.16298, -117.35695 33.1..."
1,24322,BOULEVARD EAST,BUE,138/69/12.47 kV,EA,12kV,(576 Data Points),3.70,7.30,11.01,4.48,12.0,"POLYGON ((-116.39242 32.65018, -116.39242 32.6..."
2,24323,GRANITE,GR,69/12 kV,EA,12kV,(576 Data Points),57.38,1.52,58.90,93.14,47.0,"POLYGON ((-116.87850 32.76981, -116.87856 32.7..."
3,24324,KYOCERA,KA,69/12 kV,BC,12kV,(576 Data Points),NaN,0.00,NaN,NaN,30.0,"POLYGON ((-117.14036 32.81960, -117.14036 32.8..."
4,24325,PALA,PA,69/12 kV,NE,12kV,(576 Data Points),22.68,1.34,24.02,15.91,75.0,"POLYGON ((-117.19347 33.37029, -117.19347 33.3..."


In [ ]:
# rename the substation name for consistency with other dfs
# sdge_substations = sdge_substations.rename(columns={'NAME' : 'Sub'},
#                                            inplace = True)

In [3]:
# read in the zillow geo data 
zillowmap = gpd.read_file("../../../../capstone/electrigrid/data/buildings/final_zillow.gpkg")

In [5]:
zillowmap.head()

,type,year,room,heat,cool,own,unit,value,sqft_type,sqft,ID,GEOID,p_ID,area,code,geometry
0,Multi,2003.0,1.0,None,None,I,224.0,491943.0,living,1003.0,3,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435)
1,Multi,2003.0,1.0,None,None,None,224.0,240117.0,living,936.0,4,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435)
2,Multi,2003.0,1.0,None,None,I,224.0,261770.0,living,997.0,5,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435)
3,Multi,2003.0,1.0,None,None,None,224.0,223337.0,living,1002.0,6,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435)
4,Multi,2003.0,1.0,None,None,None,224.0,241347.0,living,1003.0,7,06001403302,468,PGE/SCE,RR106,POINT (564449.237 4183218.817)


The p_ID is the substation ID. Let's join this with the data frame that has the ID and the substation name so that we have both and can easily identify whether the substation assignment matches when we map it with the San Diego substation polygons. 


In [6]:
# read in zillow csv
substation_id = pd.read_csv("../../../../capstone/electrigrid/data/buildings/zillow_cats_ica_joined.csv")

substation_id.head()

,bus_i,kV,Type,Sub,p_ID,area,GEOID,ID,X,Y
0,1.0,115.0,'Substation',jenney,h1334,Others,6.001427e+09,1746.0,-122.242977,37.772369
1,2.0,66.0,'Substation',NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,66.0,'Substation',NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,230.0,'Substation',castro valley,167,PGE/SCE,6.001431e+09,153.0,-122.061266,37.691784
4,5.0,115.0,'Substation',jarvis,67,PGE/SCE,6.001440e+09,65.0,-122.017544,37.595334


For now we just need the substation name and the substation ID. 

In [7]:
# select only the necessary columns
substation_id = substation_id[['Sub', 'p_ID']]

Now that we have a clean ID and substation data frame we can use it to verify that the substation IDs were accurately assigned.

In [8]:
zillow_subid = pd.merge(zillowmap, 
                        substation_id,
                        how = 'left', 
                        on = 'p_ID')

In [9]:
zillow_subid.head()

,type,year,room,heat,cool,own,unit,value,sqft_type,sqft,ID,GEOID,p_ID,area,code,geometry,Sub
0,Multi,2003.0,1.0,None,None,I,224.0,491943.0,living,1003.0,3,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435),oakland c
1,Multi,2003.0,1.0,None,None,I,224.0,491943.0,living,1003.0,3,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435),oakland c
2,Multi,2003.0,1.0,None,None,I,224.0,491943.0,living,1003.0,3,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435),oakland c
3,Multi,2003.0,1.0,None,None,I,224.0,491943.0,living,1003.0,3,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435),oakland c
4,Multi,2003.0,1.0,None,None,None,224.0,240117.0,living,936.0,4,06001403302,468,PGE/SCE,RR106,POINT (564447.371 4183243.435),oakland c


In [ ]:
# initial plot of san diego zillow points from the geopackage
fig, ax = plt.subplots()

sdge_substations.plot(ax=ax,
                      column = 'NAME')

zillow_subid.plot(ax = ax, 
                  column = 'Sub')

ax.set_title('San Diego Substation Polygons')
ax.axis('off')

plt.show()

The map above takes too long to run. Let's sample some points to get a better look at the data. 

In [ ]:
# how big is the dataset 
zillow_subid.shape

In [ ]:
subset_subid = zillow_subid.sample_points()